# 动机

人脸识别模型中发现保存的vgg计算的同一个图片，每次都不一样，像是随机值。用
```python
net=VGGNet()
for index,k in enumerate(net.state_dict().keys()):
    print("current model params ,layer {},key:{}, shape: {}".format(index,k, net.state_dict()[k].shape))
```
观察发现，只有最后三个全连接层的权重，其它值没保存。奇怪，和中药材识别中的vgg做个比较。

In [ ]:
import paddle
import numpy as np
import paddle.fluid as fluid

![](./image/vgg.png)

# 中药材识别中的vgg模型

In [2]:
class ConvPool(fluid.dygraph.Layer):
    '''卷积+池化'''
    def __init__(self,
                 num_channels,
                 num_filters,
                 filter_size,
                 pool_size,
                 pool_stride,
                 groups,
                 conv_stride=1,
                 conv_padding=1,
                 act=None,
                 pool_type='max'
                 ):
        super(ConvPool, self).__init__()

        self._conv2d_list = []

        for i in range(groups):
            conv2d = self.add_sublayer(   #返回一个由所有子层组成的列表。
                'bb_%d' % i,
                fluid.dygraph.Conv2D(
                    num_channels=num_channels, #通道数
                    num_filters=num_filters,   #卷积核个数
                    filter_size=filter_size,   #卷积核大小
                    stride=conv_stride,        #步长
                    padding=conv_padding,      #padding大小，默认为0
                    act=act)
            )
            num_channels = num_filters
            self._conv2d_list.append(conv2d)

        self._pool2d = fluid.dygraph.Pool2D(
            pool_size=pool_size,           #池化核大小
            pool_type=pool_type,           #池化类型，默认是最大池化
            pool_stride=pool_stride        #池化步长
        )

    def forward(self, inputs):
        x = inputs
        for conv in self._conv2d_list:
            x = conv(x)
        x = self._pool2d(x)
        return x
class VGGNet(fluid.dygraph.Layer):
    '''
    VGG网络
    '''
    def __init__(self):
        super(VGGNet, self).__init__()
        self.convpool01=ConvPool(num_channels=3,num_filters=64,filter_size=3,pool_size=2,pool_stride=2,groups=2,act='relu')
        self.convpool02=ConvPool(64,128,3,2,2,2,act='relu')
        self.convpool03=ConvPool(128,256,3,2,2,3,act='relu')
        self.convpool04=ConvPool(256,512,3,2,2,3,act='relu')
        self.convpool05=ConvPool(512,512,3,2,2,3,act='relu')
        self.pool_5_shape=512*7*7
        self.fc01=fluid.dygraph.Linear(self.pool_5_shape,4096,act='relu')
        self.fc02=fluid.dygraph.Linear(4096,4096,act='relu')
        self.fc03=fluid.dygraph.Linear(4096,128,act='softmax')

    def forward(self, inputs, label=None):
        """前向计算"""
        out=self.convpool01(inputs)
        out=self.convpool02(out)
        out=self.convpool03(out)
        out=self.convpool04(out)
        out=self.convpool05(out)
        out=fluid.layers.reshape(out,shape=[-1,self.pool_5_shape])
        out=self.fc01(out)
        out=self.fc02(out)
        out=self.fc03(out)
        if label is not None:
            acc = fluid.layers.accuracy(input=out, label=label)
            return out, acc
        else:
            return out

#vgg+dropout
class VGGNet2(fluid.dygraph.Layer):
    '''
    VGG网络
    '''
    def __init__(self):
        super(VGGNet2, self).__init__()
        self.convpool01=ConvPool(num_channels=3,num_filters=64,filter_size=3,pool_size=2,pool_stride=2,groups=2,act='relu')
        self.convpool02=ConvPool(64,128,3,2,2,2,act='relu')
        self.convpool03=ConvPool(128,256,3,2,2,3,act='relu')
        self.convpool04=ConvPool(256,512,3,2,2,3,act='relu')
        self.convpool05=ConvPool(512,512,3,2,2,3,act='relu')
        self.pool_5_shape=512*7*7

        self.fc01=fluid.dygraph.Linear(self.pool_5_shape,4096,act='relu')
        self.fc02=fluid.dygraph.Linear(4096,4096,act='relu')
        self.fc03=fluid.dygraph.Linear(4096,128,act='softmax')

    def forward(self, inputs, label=None):
        """前向计算"""
        out=self.convpool01(inputs)
        out=self.convpool02(out)
        out=self.convpool03(out)
        out=self.convpool04(out)
        out=self.convpool05(out)
        out=fluid.layers.reshape(out,shape=[-1,self.pool_5_shape])

        drop = fluid.layers.dropout(x=out, dropout_prob=0.5)
        out=self.fc01(drop)
        bn = fluid.layers.batch_norm(input=out, act='relu')
        drop2 = fluid.layers.dropout(x=bn, dropout_prob=0.5)
        out=self.fc02(drop2)
        out=self.fc03(out)

        if label is not None:
            acc = fluid.layers.accuracy(input=out, label=label)
            return out, acc
        else:
            return out

## 查看层数

保存模型后参数大小为700M左右

In [17]:
net=VGGNet()
for index,k in enumerate(net.state_dict().keys()):
    print("current model params ,layer {},key:{}, shape: {}".format(index,k, net.state_dict()[k].shape))
with fluid.dygraph.guard():
    fluid.dygraph.save_dygraph(net.state_dict(),'vgg_self2')

current model params ,layer 0,key:convpool01.bb_0.weight, shape: [64, 3, 3, 3]
current model params ,layer 1,key:convpool01.bb_0.bias, shape: [64]
current model params ,layer 2,key:convpool01.bb_1.weight, shape: [64, 64, 3, 3]
current model params ,layer 3,key:convpool01.bb_1.bias, shape: [64]
current model params ,layer 4,key:convpool02.bb_0.weight, shape: [128, 64, 3, 3]
current model params ,layer 5,key:convpool02.bb_0.bias, shape: [128]
current model params ,layer 6,key:convpool02.bb_1.weight, shape: [128, 128, 3, 3]
current model params ,layer 7,key:convpool02.bb_1.bias, shape: [128]
current model params ,layer 8,key:convpool03.bb_0.weight, shape: [256, 128, 3, 3]
current model params ,layer 9,key:convpool03.bb_0.bias, shape: [256]
current model params ,layer 10,key:convpool03.bb_1.weight, shape: [256, 256, 3, 3]
current model params ,layer 11,key:convpool03.bb_1.bias, shape: [256]
current model params ,layer 12,key:convpool03.bb_2.weight, shape: [256, 256, 3, 3]
current model par

# 人脸识别中的vgg网络

In [20]:
import paddle
import paddle.fluid as fluid
from paddle.fluid import Conv2D, Pool2D, Linear,BatchNorm,Dropout

class conv_bn(fluid.dygraph.Layer):
    def __init__(self,name,in_channels,out_channels,act=None):
        super(conv_bn,self).__init__(name)
        name=self.full_name()
        self.conv = Conv2D(num_channels=in_channels, num_filters=out_channels,filter_size=3, padding=1, act=None,bias_attr=False)
        self.bn=BatchNorm(num_channels=out_channels,act=act)

    def forward(self,x):
        return self.bn(self.conv(x))

class vgg_block(fluid.dygraph.Layer):
    def __init__(self, name, in_channels, out_channels, groups):
        super(vgg_block, self).__init__(name)
        name = self.full_name()
        self.layers = []
        for i in range(groups):
            conv_layer = conv_bn(name=name + '_conv_bn_' + str(i),in_channels=in_channels, out_channels=out_channels,act='relu')
            self.layers.append(conv_layer)
            in_channels = out_channels
        self.pool = Pool2D(pool_size=2, pool_stride=2, pool_type='max')

    def forward(self, x):
        for layer in self.layers:
            #print(layer.full_name())
            x = layer(x)
        return self.pool(x)
class vgg_face_question(fluid.dygraph.Layer):
    def __init__(self):
        super(vgg_face_question, self).__init__()
        self.layers = []
        params = ((3, 64, 2), (64, 128, 2), (128, 256, 3), (256, 512, 3), (512, 512, 3))
        for in_c, out_c, groups in params:
            self.layers.append(self.add_sublayer(name='vgg_block' + str(in_c) + "_" + str(out_c),sublayer=vgg_block(name='vgg_block' + str(in_c) + "_" + str(out_c), in_channels=in_c, out_channels=out_c,groups=groups)))
        self.fc1 = Linear(input_dim=512 * 7 * 7, output_dim=4096, act='relu')
        self.fc2 = Linear(input_dim=4096, output_dim=1024, act='relu')
        self.fc3 = Linear(input_dim=1024, output_dim=128)

    def forward_before(self, x):
        for layer in self.layers:
            x = layer(x)
        x=fluid.layers.reshape(x,[x.shape[0],-1])
        x=self.fc1(x)
        x=self.fc2(x)
        x=self.fc3(x)
        return x
    def forward(self, img1,img2):
        return self.forward_before(img1),self.forward_before(img2)

In [23]:
net=vgg_face_question()
for index,k in enumerate(net.state_dict().keys()):
    print("current model params ,layer {},key:{}, shape: {}".format(index,k, net.state_dict()[k].shape))
with fluid.dygraph.guard():
    fluid.dygraph.save_dygraph(net.state_dict(),'vgg_face_question')

current model params ,layer 0,key:fc1.weight, shape: [25088, 4096]
current model params ,layer 1,key:fc1.bias, shape: [4096]
current model params ,layer 2,key:fc2.weight, shape: [4096, 1024]
current model params ,layer 3,key:fc2.bias, shape: [1024]
current model params ,layer 4,key:fc3.weight, shape: [1024, 128]
current model params ,layer 5,key:fc3.bias, shape: [128]


# 问题

以上运行只有6层参数：
```plain
current model params ,layer 0,key:fc1.weight, shape: [25088, 4096]
current model params ,layer 1,key:fc1.bias, shape: [4096]
current model params ,layer 2,key:fc2.weight, shape: [4096, 1024]
current model params ,layer 3,key:fc2.bias, shape: [1024]
current model params ,layer 4,key:fc3.weight, shape: [1024, 128]
current model params ,layer 5,key:fc3.bias, shape: [128]
```
自己计算下参数大概400M左右，实际上599M。
问题在vgg_block循环增加那里
```python
 conv_layer = conv_bn(name=name + '_conv_bn_' + str(i),in_channels=in_channels, out_channels=out_channels,act='relu')
```
应该是
```python
conv_layer = self.add_sublayer(name='_conv_bn_' + str(i),sublayer=conv_bn(name=name + '_conv_bn_' + str(i),in_channels=in_channels, out_channels=out_channels,act='relu'))
```
# 修改后的人脸识别VGG


In [25]:
import paddle
import paddle.fluid as fluid
from paddle.fluid import Conv2D, Pool2D, Linear,BatchNorm,Dropout

class conv_bn(fluid.dygraph.Layer):
    def __init__(self,name,in_channels,out_channels,act=None):
        super(conv_bn,self).__init__(name)
        name=self.full_name()
        self.conv = Conv2D(num_channels=in_channels, num_filters=out_channels,filter_size=3, padding=1, act=None,bias_attr=False)
        self.bn=BatchNorm(num_channels=out_channels,act=act)

    def forward(self,x):
        return self.bn(self.conv(x))

class vgg_block(fluid.dygraph.Layer):
    def __init__(self, name, in_channels, out_channels, groups):
        super(vgg_block, self).__init__(name)
        name = self.full_name()
        self.layers = []
        for i in range(groups):
            conv_layer = self.add_sublayer(name='_conv_bn_' + str(i),sublayer=conv_bn(name=name + '_conv_bn_' + str(i),in_channels=in_channels, out_channels=out_channels,act='relu'))
            self.layers.append(conv_layer)
            in_channels = out_channels
        self.pool = Pool2D(pool_size=2, pool_stride=2, pool_type='max')

    def forward(self, x):
        for layer in self.layers:
            #print(layer.full_name())
            x = layer(x)
        return self.pool(x)

class vgg_face(fluid.dygraph.Layer):
    def __init__(self):
        super(vgg_face, self).__init__()
        self.layers = []
        params = ((3, 64, 2), (64, 128, 2), (128, 256, 3), (256, 512, 3), (512, 512, 3))
        for in_c, out_c, groups in params:
            self.layers.append(self.add_sublayer(name='vgg_block' + str(in_c) + "_" + str(out_c),sublayer=vgg_block(name='vgg_block' + str(in_c) + "_" + str(out_c), in_channels=in_c, out_channels=out_c,groups=groups)))
        self.fc1 = Linear(input_dim=512 * 7 * 7, output_dim=4096, act='relu')
        self.fc2 = Linear(input_dim=4096, output_dim=1024, act='relu')
        self.fc3 = Linear(input_dim=1024, output_dim=128)

    def forward_before(self, x):
        for layer in self.layers:
            x = layer(x)
        x=fluid.layers.reshape(x,[x.shape[0],-1])
        x=self.fc1(x)
        x=self.fc2(x)
        x=self.fc3(x)
        return x
    def forward(self, img1,img2):
        return self.forward_before(img1),self.forward_before(img2)

In [26]:
net=vgg_face()
for index,k in enumerate(net.state_dict().keys()):
    print("current model params ,layer {},key:{}, shape: {}".format(index,k, net.state_dict()[k].shape))
with fluid.dygraph.guard():
    fluid.dygraph.save_dygraph(net.state_dict(),'vgg_face')

current model params ,layer 0,key:vgg_block3_64._conv_bn_0.conv.weight, shape: [64, 3, 3, 3]
current model params ,layer 1,key:vgg_block3_64._conv_bn_0.bn.weight, shape: [64]
current model params ,layer 2,key:vgg_block3_64._conv_bn_0.bn.bias, shape: [64]
current model params ,layer 3,key:vgg_block3_64._conv_bn_0.bn._mean, shape: [64]
current model params ,layer 4,key:vgg_block3_64._conv_bn_0.bn._variance, shape: [64]
current model params ,layer 5,key:vgg_block3_64._conv_bn_1.conv.weight, shape: [64, 64, 3, 3]
current model params ,layer 6,key:vgg_block3_64._conv_bn_1.bn.weight, shape: [64]
current model params ,layer 7,key:vgg_block3_64._conv_bn_1.bn.bias, shape: [64]
current model params ,layer 8,key:vgg_block3_64._conv_bn_1.bn._mean, shape: [64]
current model params ,layer 9,key:vgg_block3_64._conv_bn_1.bn._variance, shape: [64]
current model params ,layer 10,key:vgg_block64_128._conv_bn_0.conv.weight, shape: [128, 64, 3, 3]
current model params ,layer 11,key:vgg_block64_128._conv_bn